# Fisher FIFO

In the `fisher-fifo-v3` notebook, we set a code to use the Fisher information effectively in neural netwrks using the *first-in-first-out* strategy to store gradients.

Here our objective is to assess the effect of the FIFO buffer in the quality of the model. Also, we implement the partitioning strategy to make our algorithm generalizable to larger networks, as well as datasets with larger instances (like images).

To enhance the partition effectiveness, we proceed to use the "maximum-block-update", to make the algorithm faster.

---

in fisher `fisher-fifo-v4` notebook we are trying to implement the algorithm in a more efficient way by using the [torch.bmm](https://pytorch.org/docs/stable/generated/torch.bmm.html). The main idea is to execute the matrix-multiplications of more than one block in an optimized way.

---

in `fisher-fifo-v4.2` notebook, we are trying to make the algorithm even faster using a single centralized object responsible for storing all the matrices (and their inverses) as well as all gradients and buffers. The main idea here is to make the most of vectorization using Pytorch utilities for GPU.

---

in `fisher-fifo-v4.3` notebook, we try to take the algorithm one step further in efficiency. We are implementing the strategy to retrieve the partitions in sets, instead of individually.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import math
import os
import json

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skopt import gp_minimize

from scipy import stats

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/fisher-fifo-v4-3-comparing-cifar100-resnet18/bayes-opt-results.npz
/kaggle/input/fisher-fifo-v4-3-comparing-cifar100-resnet18/results_step_20.json
/kaggle/input/fisher-fifo-v4-3-comparing-cifar100-resnet18/__results__.html
/kaggle/input/fisher-fifo-v4-3-comparing-cifar100-resnet18/__notebook__.ipynb
/kaggle/input/fisher-fifo-v4-3-comparing-cifar100-resnet18/__output__.json
/kaggle/input/fisher-fifo-v4-3-comparing-cifar100-resnet18/custom.css
/kaggle/input/fisher-fifo-v4-3-comparing-cifar100-resnet18/data/cifar-100-python.tar.gz
/kaggle/input/fisher-fifo-v4-3-comparing-cifar100-resnet18/data/cifar-100-python/meta
/kaggle/input/fisher-fifo-v4-3-comparing-cifar100-resnet18/data/cifar-100-python/file.txt~
/kaggle/input/fisher-fifo-v4-3-comparing-cifar100-resnet18/data/cifar-100-python/test
/kaggle/input/fisher-fifo-v4-3-comparing-cifar100-resnet18/data/cifar-100-python/train


In [3]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

import torchvision
import torchvision.datasets as datasets

In [4]:
def get_device():    
    if torch.cuda.is_available():

        device = torch.device('cuda')
        print( torch.cuda.get_device_name(device) )
        print( torch.cuda.get_device_properties(device) )

    else:
        device = torch.device('cpu')
        print(device)
        
    return device

In [5]:
!pip install torchsummary
import torchsummary

In [6]:
class cfg:
    # n_features = 28 * 28
    img_size = (32, 32)
    img_channels = 3
    n_classes = 100  ## we have 100 classes in CIFAR100
    
    # device = torch.device('cpu')
    device = get_device()
    
    max_loss = 20.0

Tesla P100-PCIE-16GB
_CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


# create the dataset

In [7]:
def generate_dataset_mnist(batch_size):
    print(f'generating MNIST data with {cfg.n_classes} classes')
    
    transf_ = torchvision.transforms.Compose([
        # torchvision.transforms.Resize(size=[14, 14]),
        torchvision.transforms.ToTensor()
    ])
    
    mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transf_)
    mnist_test  = datasets.MNIST(root='./data', train=False, download=True, transform=transf_)
    
    mnist_train_dataloader = DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True)
    mnist_test_dataloader  = DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=False)

    return mnist_train_dataloader, mnist_test_dataloader

In [8]:
def generate_dataset_cifar10(batch_size):
    print(f'generating CIFAR10 data with {cfg.n_classes} classes')
    
    transf_ = torchvision.transforms.Compose([
        # torchvision.transforms.Resize(size=[14, 14]),
        torchvision.transforms.ToTensor()
    ])
    
    cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transf_)
    cifar10_test  = datasets.CIFAR10(root='./data', train=False, download=True, transform=transf_)
    
    cifar10_train_dataloader = DataLoader(dataset=cifar10_train, batch_size=batch_size, shuffle=True)
    cifar10_test_dataloader  = DataLoader(dataset=cifar10_test, batch_size=batch_size, shuffle=False)

    return cifar10_train_dataloader, cifar10_test_dataloader

In [9]:
def generate_dataset_cifar100(batch_size):
    print(f'generating CIFAR100 data with {cfg.n_classes} classes')
    
    transf_ = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()
    ])
    
    cifar100_train = datasets.CIFAR100(root='./data', train=True, download=True, transform=transf_)
    cifar100_test  = datasets.CIFAR100(root='./data', train=False, download=True, transform=transf_)
    
    cifar100_train_dataloader = DataLoader(dataset=cifar100_train, batch_size=batch_size, shuffle=True)
    cifar100_test_dataloader  = DataLoader(dataset=cifar100_test, batch_size=batch_size, shuffle=False)

    return cifar100_train_dataloader, cifar100_test_dataloader

## declaring network architecture

In [10]:
def get_default_network(c=16, device=cfg.device):
    net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=cfg.n_features, out_features=c),
        nn.ReLU(),
        nn.Linear(in_features=c, out_features=c),
        nn.ReLU(),
        nn.Linear(in_features=c, out_features=c),
        nn.ReLU(),
        nn.Linear(in_features=c, out_features=cfg.n_classes)
    )
    
    torchsummary.summary(net, input_size=[[cfg.n_features]], device='cpu')
    
    return net

In [11]:
def get_cnn_network(in_channels=cfg.img_channels, c=16, p_drop=0.1, device=cfg.device):
    
    img_flat_size = (4 * c * (cfg.img_size[0] // 8) * (cfg.img_size[1] // 8) )
    print(img_flat_size)
    net = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=c, kernel_size=5, stride=2, padding=2),
        nn.ReLU(),

        nn.Conv2d(in_channels=c, out_channels=(2 * c), kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Dropout2d(p=p_drop),
        
        nn.Conv2d(in_channels=(2 * c), out_channels=(4 * c), kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Dropout2d(p=p_drop),
        
        nn.Flatten(),
        
        nn.Linear(in_features=img_flat_size, out_features=(8 * c) ),
        nn.ReLU(),
        nn.Dropout(p=p_drop),
        
        nn.Linear(in_features=(8 * c), out_features=(4 * c) ),
        nn.ReLU(),
        nn.Dropout(p=p_drop),
        
        nn.Linear(in_features=(4 * c), out_features=cfg.n_classes)
    )
    
    torchsummary.summary(net, input_size=[[cfg.img_channels, *cfg.img_size]], device='cpu')
    
    return net

In [12]:
def get_cnn_network_v2(in_channels=cfg.img_channels, p_drop=0.1, device=cfg.device):
    
    net = nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=96, kernel_size=5, padding=2),
        nn.MaxPool2d(kernel_size=2),
        nn.ReLU(),

        nn.Conv2d(in_channels=96, out_channels=80, kernel_size=5, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Dropout2d(p=p_drop),
        
        nn.Conv2d(in_channels=80, out_channels=96, kernel_size=5, padding=2),
        nn.ReLU(),
        nn.Dropout2d(p=p_drop),
        
        nn.Conv2d(in_channels=96, out_channels=64, kernel_size=5, padding=2),
        nn.ReLU(),
        nn.Dropout2d(p=p_drop),
        
        nn.Flatten(),
        
        # nn.Linear(in_features=4096, out_features=256 ),
        nn.Linear(in_features=(cfg.img_size[0] // 4) * (cfg.img_size[1] // 4) * 64, out_features=256 ),
        nn.ReLU(),
        nn.Dropout(p=p_drop),
        
        nn.Linear(in_features=256, out_features=cfg.n_classes)
    )
    
    torchsummary.summary(net, input_size=[[cfg.img_channels, *cfg.img_size]], device='cpu')
    
    return net

In [13]:
def get_resnet18(device=cfg.device):
    
    net = torchvision.models.resnet18(num_classes=cfg.n_classes)
    torchsummary.summary(net, input_size=[[cfg.img_channels, *cfg.img_size]], device='cpu')
    
    return net

# object for calculation of the metrics

In [14]:
class Metrics():
    def __init__(self, value_round=None, time_round=None):
        self.metrics_dict = {}
        self.set_initial_time()
        self.val_round = value_round
        self.time_round = time_round
        
    def set_initial_time(self):
        self.init_time = time.time()
        
    def get_time(self):
        return time.time() - self.init_time
    
    def add(self, key, value, step=None):
        
        if step is None:
            step = np.nan
        
        if key not in self.metrics_dict:
            self.metrics_dict[key] = []
        
        t = self.get_time()
        if self.time_round is not None:
            t = round(t, ndigits=self.time_round)
        
        if self.val_round is not None:
            value = round(value, ndigits=self.val_round)
        
        self.metrics_dict[key].append( (value, step, t) )
    
    def add_(self, dict_, step=None):
        for key, value in dict_.items():
            self.add(key, value, step)
    
    def get(self, key, get_step=False, get_time=False):
        y, x, t = zip(*self.metrics_dict[key])
        y, x, t = list(y), list(x), list(t)
        
        return x, y, t

# Fisher Information calculation objects

In [15]:
class FisherFIFO():
    def __init__(self,
                 named_params,
                 buffer_size,
                 partition_size,
                 block_updates):
        
        self.buffer_size = buffer_size
        self.partition_size = partition_size
        self.block_updates = block_updates
        
        named_params = list(named_params)
        
        self.partition_fisher_list = []
        total_partitions, total_block_upd = 0, 0
        for pi, (n, p) in enumerate( named_params ):
            part_fisher = PartitionerFisherFIFO(param = p,
                                                name = n,
                                                buffer_size = buffer_size,
                                                partition_size = partition_size,
                                                block_updates = block_updates,
                                                parent_fifo = None)
            
            self.partition_fisher_list.append( (p, part_fisher, total_partitions, total_block_upd) )
            
            total_partitions += part_fisher.num_part
            total_block_upd += part_fisher.block_updates
            
        self.num_part = total_partitions
        self.total_block_updates = total_block_upd
        
        print(f'total partitions: {self.num_part} - effective block updates: {self.total_block_updates}')
        
        ## pre-alocate the memory for the tensor that stores the selected gradients (changes every iteration)
        self.g = torch.zeros(size=[self.total_block_updates, partition_size, 1], dtype=torch.float, device=cfg.device)
        
        ## pre-alocate the memory for the tensor that stores the buffer and the tensor for the inverse
        self.buffer = torch.zeros(size=[self.num_part, partition_size, buffer_size], dtype=torch.float, device=cfg.device)
        self.fisher_inv = torch.zeros(size=[self.num_part, partition_size, partition_size], dtype=torch.float, device=cfg.device)        
    
        print('initializing buffers and inverses...')
        ## now we initialize the buffer and the inverse for all partitions
        i = 0
        for _, part_fisher, _, _ in self.partition_fisher_list:
            for _, start, end in part_fisher.ind_fisher_list:

                if i == 0 or ( (i + 1) % 10000 ) == 0 or i == (self.num_part - 1):
                    print(f'partition {i+1}/{self.num_part}')

                n = end - start
                buffer, _, fisher_inv = self.initialize_fisher_partition(param_size=n, buffer_size=self.buffer_size)

                self.buffer[i, :n, :] = buffer
                self.fisher_inv[i, :n, :n] = fisher_inv

                i += 1

    
    def initialize_fisher_partition(self, param_size, buffer_size):
        
        buffer = self.get_initial_buffer_v2(param_size, buffer_size)
            
        ## shuffle buffer across columns
        buffer = buffer[:, torch.randperm(buffer.shape[1]) ]
        
        ## the fisher matrix will be initialized as G @ G.T, in which G is our buffer. We built
        ## our buffer in a smart way so the resulting Fisher info matrix is initialized close to identity
        fisher = buffer @ buffer.T
        
        ## since our Fisher information is diagonal for now, its inverse is given just by the innverted
        ## elements of the diagonal. 
        fisher_inv = torch.diag( 1 / torch.diag(fisher) )
        
        return buffer, fisher, fisher_inv
    
    
    def get_initial_buffer_v2(self, param_size, buffer_size):
        ## here we adopt a faster approach to initialize the buffer. We use n 
        ## identity matrices concatenated column-wise. n is determined by `param_size` and `buffer_size`
        
        n_eye = math.ceil(buffer_size / param_size)
        I = torch.eye(n=param_size, dtype=torch.float, device=cfg.device)
        
        buffer = torch.cat(n_eye * [I], dim=1)[:, :buffer_size]
        
        assert buffer.shape == (param_size, buffer_size)
        
        return buffer


    def get_idx_lists(self):
        run_enc_list = []
        default_idx_list = []
        for p, part_fisher, num_part, block_upd in self.partition_fisher_list:
            init_block, end_block, g_init_idx, g_end_idx = part_fisher.get_random_blocks()
            
            # print(f'param shape: {p.shape} - blocks: {init_block} to {end_block} - grad: {g_init_idx} to {g_end_idx}')
            
            run_enc_list.append( (num_part + init_block, num_part + end_block, g_init_idx, g_end_idx) )
            default_idx_list.append( np.arange(start=num_part + init_block, stop=num_part + end_block + 1) )
            
            
        return run_enc_list, np.concatenate(default_idx_list)
    
    
    def read_gradients(self, idx):
        self_g_start = 0
        for i, (_, _, g_start, g_end) in enumerate(idx):
            n_grad = g_end - g_start
            # self_g_end = min( self_g_start + n_grad, torch.numel(self.g) )
            self_g_end = self_g_start + n_grad
            
            p, _, _, _ = self.partition_fisher_list[i]
            
            self.g.view(-1)[self_g_start:self_g_end] = p.grad.view(-1)[g_start:g_end]
            
            if (n_grad % self.partition_size) > 0:
                extra_zeros = self.partition_size - (n_grad % self.partition_size)
                self.g.view(-1)[self_g_end:(self_g_end + extra_zeros)] = 0.0
            else:
                extra_zeros = 0
            
#             print(f'self_g_start: {self_g_start} - self_g_end: {self_g_end} - self.g.shape: {self.g.view(-1).shape}')
#             print(f'g_start: {g_start} - g_end: {g_end} - p.grad.shape: {p.grad.view(-1).shape}')
#             print(f'n_grad: {n_grad} - part-size: {self.partition_size} - extra-zeros: {extra_zeros}')
#             print()
            
            self_g_start = self_g_end + extra_zeros
            

    def write_gradients(self, idx):
        self_g_start = 0
        for i, (_, _, g_start, g_end) in enumerate(idx):
            n_grad = g_end - g_start
            self_g_end = self_g_start + n_grad
            
            p, _, _, _ = self.partition_fisher_list[i]
            p.grad.view(-1)[g_start:g_end] = self.g.view(-1)[self_g_start:self_g_end]

            if (n_grad % self.partition_size) > 0:
                extra_zeros = self.partition_size - (n_grad % self.partition_size)
            else:
                extra_zeros = 0
            
            self_g_start = self_g_end + extra_zeros

    
    def step(self):
        ## selects the blocks to be updated
        run_enc_idx, default_idx = self.get_idx_lists()
        
        ## read the selected blocks gradients and stores them in self.g
        self.read_gradients(run_enc_idx)
        
        ## get apart the inverses and buffers for the selected blocks
        inv = self.fisher_inv[default_idx, ...]
        buffer = self.buffer[default_idx, ...]
        
        # print(inv.shape, buffer.shape)
        
        ## update the buffer
        
        ## dimensions are: partitions, gradient-size, bufffer-size
        g_old = buffer[:, :, 0:1] 
        
        ## we  update in the third dimension: "buffer-size"
        buffer = torch.cat([buffer[:, :, 1:], self.g], dim=2)
        
        ## update the inverses and modify current gradients
        ## ...
        sqrt_NB = math.sqrt(self.buffer_size)
        
        ## update inverse - phase 1: add new gradient ##
        g_phase1 = self.upd_inverse( (1 / sqrt_NB) * self.g, inv, type_='sum')

        ## update inverse - phase 2: remove old gradient ##
        self.upd_inverse( (1 / sqrt_NB) * g_old, inv, type_='sub')

        ## modify the current gradients
        if False:
            ## use the "phase-1-trick" to get the estimated new gradient
            self.g = g_phase1 * sqrt_NB
        else:
            ## get the modified gradient using "de facto" the new inverses and the gradients
            self.g = self.modify_grad(self.g, inv)
        
        ## return the inverses and buffers to the main tensor
        self.fisher_inv[default_idx, ...] = inv
        self.buffer[default_idx, ...] = buffer
        
        ## return the modified gradients to the parameters
        self.write_gradients(run_enc_idx)


    def upd_inverse(self, g, inverse, type_='sum'):
        ## update the inverse based on the woodbury inversion
        f_inv_g = torch.bmm(inverse, g)

        if type_ == 'sum':
            d = 1 + torch.sum(g * f_inv_g, dim=[1, 2], keepdim=True)
            inverse[:] = inverse - (f_inv_g * torch.transpose(f_inv_g, 1, 2) / d)

        elif type_ == 'sub':
            d = 1 - torch.sum(g * f_inv_g, dim=[1, 2], keepdim=True)
            inverse[:] = inverse + (f_inv_g * torch.transpose(f_inv_g, 1, 2) / d)

        else:
            ## incorrect type
            print('incorrect rank-1 update type: ' + type_)
        
        return f_inv_g


    def modify_grad(self, g, inverse):
        return torch.bmm(inverse, g)

In [16]:
class PartitionerFisherFIFO():
    def __init__(self,
                 param,
                 name,
                 buffer_size,
                 partition_size,
                 block_updates,
                 parent_fifo):
        
        self.param = param
        self.name = name 
        
        if partition_size is None:
            self.partition_size = param.numel()
        else:
            self.partition_size = partition_size
        
        ## calculates the number of partitions required. It is calculated using the param size and
        ## our partition maximum size. The gradient (the same size as param) is going to be partitioned in
        ## equal pieces (except possibly the last one) to be processed individually by our "IndividualFisherFIFO"
        self.param_size = param.numel()
        self.num_part = math.ceil(self.param_size / self.partition_size)
        
        ## the number of blocks (partitions) to update at each iteration. This can be < num_part to make
        ## the algorithm more efficient. (we dont update every partition at every iteration)
        if block_updates is None:
            self.block_updates = self.num_part
        else:
            self.block_updates = min(block_updates, self.num_part)
        
        print(f'FisherPartitioner: param: {self.param_size} - partition: {self.partition_size} - nº part: {self.num_part} - block updates: {self.block_updates}')
                
        ## the list stores the indexes used to partition the gradient
        self.ind_fisher_list = []
        for i in range(self.num_part):
            start = i * self.partition_size
            end = min(start + self.partition_size, self.param_size)
            
            self.ind_fisher_list.append( (i, start, end) )
        
    
    def get_random_blocks(self, num_part=None, block_upd=None):
        
        if num_part is None:
            num_part = self.num_part
        
        if block_upd is None:
            block_upd = self.block_updates
        
        ## choose the initial block randomly
        init_block = np.random.choice(num_part - block_upd + 1)
        
        ## the final block will be necessarily `block_upd` blocks further. This means we select
        ## a contiguous sequence of blocks. This is going to be used for performance reasons
        end_block = init_block + block_upd - 1
        
        ## therefore, the starting and ending index to be used to fetch the gradient positions for the
        ## blocks will be the starting index for the first block and the ending positions for the last block
        _, g_init_idx, _ = self.ind_fisher_list[init_block]
        _, _, g_end_idx = self.ind_fisher_list[end_block]
        
        return init_block, end_block, g_init_idx, g_end_idx

---

# utils function for training

In [17]:
def accuracy_score_tns(y_true, y_pred):
    return torch.mean( (y_true == y_pred).to(dtype=torch.float) ).cpu().item()

In [18]:
def train_iteration(x, y, net, optim, loss, fisher=None):
    net.train()
    net.zero_grad()
    
    y_pred = net(x)
    l = loss(y_pred, y)
    
    l.backward()
    
    if fisher is not None:
        fisher.step()
    
    optim.step()
    
    return l.item(), accuracy_score_tns( y.view(-1), y_pred.argmax(dim=1).view(-1) )

In [19]:
def evaluate(net, dataloader, loss):
    net.eval()
    
    with torch.no_grad():

        loss_list = []
        y_pred_list = []
        y_label_list = []
        for x, y in dataloader:
            
            x = x.to(cfg.device)
            y = y.to(cfg.device)

            y_pred = net(x)
            l = loss(y_pred, y)

            loss_list.append( l.cpu().item() )
            y_pred_list.append( y_pred.argmax(dim=1).view(-1) )
            y_label_list.append( y.view(-1) )

        y_pred_list = torch.cat(y_pred_list).view(-1)
        y_label_list = torch.cat(y_label_list).view(-1)

    return np.mean(loss_list), accuracy_score_tns(y_label_list, y_pred_list)

# training

In [20]:
def train_network_fisher_optimization(batch_size = 32,
                                      lr = 1e-3,
                                      momentum = 0.9,
                                      epochs = 30,
                                      buffer_size = 1000,
                                      partition_size = 256,
                                      block_updates = 4,
                                      net_params = {'c':16, 'p':0.1},
                                      apply_fisher = True,
                                      # gpu_memory_check = 20,
                                      time_limit_secs = 600,
                                      interval_print = 100):

    ## declare (instantiate) the dataset
    # train_dataloader, test_dataloader = generate_dataset_cifar10(batch_size = batch_size)
    # train_dataloader, test_dataloader = generate_dataset_mnist(batch_size = batch_size)
    train_dataloader, test_dataloader = generate_dataset_cifar100(batch_size = batch_size)

    ## instantiate the network
    # net = get_cnn_network_v2(p_drop = net_params['p']).to(device=cfg.device)
    net = get_resnet18().to(device=cfg.device)
    
    if apply_fisher:
        ## instantiate FisherFIFO object to create and update the Fisher info matrix
        fisher_fifo = FisherFIFO(named_params = net.named_parameters(),
                                 buffer_size = buffer_size,
                                 partition_size = partition_size,
                                 block_updates = block_updates)
    else:
        fisher_fifo = None

    ## create loss object: we multiply by our constant to stabilize norms
    # cross_entropy = nn.CrossEntropyLoss(reduction='mean') # standard version
    cross_entropy_standard = nn.CrossEntropyLoss(reduction='mean')
    cross_entropy = lambda y_pred, y: math.sqrt(batch_size) * cross_entropy_standard(y_pred, y)
    
    ## create optimize objects
    optim = torch.optim.SGD(params=net.parameters(), lr=lr, momentum=momentum)

    default_metrics = Metrics(value_round=3, time_round=2)

    ini_time = time.time()

    step = 0
    training_finished = False
    for epc in range(1, epochs + 1):
        
        if training_finished:
            break
        
        print(f'starting epoch: {epc}/{epochs}')

        for nbt, (x, y) in enumerate(train_dataloader):

            if training_finished:
                break

            x = x.to(cfg.device)
            y = y.to(cfg.device)

            train_loss, train_acc = train_iteration(x, y, net, optim, cross_entropy, fisher_fifo)
            default_metrics.add_({'train-loss': train_loss, 'train-acc': train_acc}, step=step)
            
            ## check time limit
            t = int(time.time() - ini_time)
            if t > time_limit_secs:
                print('time is up! finishing training')
                training_finished = True

            if ( (nbt + 1) % interval_print ) == 0 or (nbt + 1) == len(train_dataloader) or training_finished:
                avg_train_loss = np.mean( default_metrics.get('train-loss')[1][-interval_print:] )
                avg_train_acc = np.mean( default_metrics.get('train-acc')[1][-interval_print:] )
                
                test_loss, test_acc = evaluate(net, test_dataloader, cross_entropy)
                default_metrics.add_({'test-loss': test_loss, 'test-acc': test_acc}, step=step)

                m, s = t // 60, t % 60

                print(f'batch: {nbt + 1}/{len(train_dataloader)}', end='')
                print(f' - train loss: {avg_train_loss:.4f} - test loss: {test_loss:.4f}', end='')
                print(f' - train acc: {avg_train_acc:.4f} - test acc: {test_acc:.4f}', end='')
                print(f' - {m}m {s}s')
                
            step += 1

        ## check for GPU memory consumption
        if torch.cuda.is_available():
            mem_alloc_gb = torch.cuda.memory_allocated(cfg.device) / 1024**3
            mem_res_gb = torch.cuda.memory_reserved(cfg.device) / 1024**3
            max_mem_alloc_gb = torch.cuda.max_memory_allocated(cfg.device) / 1024**3
            max_mem_res_gb = torch.cuda.max_memory_reserved(cfg.device) / 1024**3

            print(f'GPU memory used: {mem_alloc_gb:.2f} GB - max: {max_mem_alloc_gb:.2f} GB - memory reserved: {mem_res_gb:.2f} GB - max: {max_mem_res_gb:.2f} GB')

            # torch.cuda.empty_cache()

    return default_metrics, fisher_fifo

In [21]:
# train_network_fisher_optimization(apply_fisher = True,
#                                    buffer_size = 32,
#                                    partition_size = 16,
#                                    block_updates = 100,
#                                    net_params = {'p': 0.1},
#                                    epochs = 10,
#                                    time_limit_secs = 15 * 60)

In [22]:
# net = torchvision.models.resnet18()
# torchsummary.summary(net, input_size=[[3, 64, 64]], device='cpu')

In [23]:
last_step_saved = None

def results_list_to_json(results_list, out_dir='/kaggle/working', step=0):
    global last_step_saved

    json_results = []

    for metrics, bs, ps, bu in results_list:
        json_results.append({
            'buffer-size': bs,
            'partition-size': ps,
            'blocks-updates': bu,
            'metrics': metrics.metrics_dict
        })

    with open( os.path.join(out_dir, f'results_step_{step}.json'), 'w' ) as fp:
        json.dump(json_results, fp)
    
    if last_step_saved is not None:
        old_file = os.path.join(out_dir, f'results_step_{last_step_saved}.json')
        if os.path.exists(old_file):
            os.remove(old_file)
    
    last_step_saved = step

In [24]:
def get_min_test_loss(metrics):
    _, test_loss, _ = metrics.get('test-loss')
    return min(test_loss)

---

In [25]:
nruns = 20

In [26]:
results_list = []
step_i = 0

buffer_size = 140
partition_size = 30
block_updates = 360

for _ in range(nruns):

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    print(f'testing - buf size: {buffer_size} - part size: {partition_size} - block upd: {block_updates} - combination nº: {step_i + 1}')

    default_metrics, _ = train_network_fisher_optimization(apply_fisher = True,
                                                           buffer_size = buffer_size,
                                                           partition_size = partition_size,
                                                           block_updates = block_updates,
                                                           net_params = {'p': 0.1},
                                                           epochs = 100,
                                                           time_limit_secs = 1200)

    results_list.append( (default_metrics, buffer_size, partition_size, block_updates) )
    results_list_to_json(results_list, step=step_i)
    step_i += 1
    
    print()

testing - buf size: 140 - part size: 30 - block upd: 360 - combination nº: 1
generating CIFAR100 data with 100 classes


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNo

initializing buffers and inverses...
partition 1/374302
partition 10000/374302
partition 20000/374302
partition 30000/374302
partition 40000/374302
partition 50000/374302
partition 60000/374302
partition 70000/374302
partition 80000/374302
partition 90000/374302
partition 100000/374302
partition 110000/374302
partition 120000/374302
partition 130000/374302
partition 140000/374302
partition 150000/374302
partition 160000/374302
partition 170000/374302
partition 180000/374302
partition 190000/374302
partition 200000/374302
partition 210000/374302
partition 220000/374302
partition 230000/374302
partition 240000/374302
partition 250000/374302
partition 260000/374302
partition 270000/374302
partition 280000/374302
partition 290000/374302
partition 300000/374302
partition 310000/374302
partition 320000/374302
partition 330000/374302
partition 340000/374302
partition 350000/374302
partition 360000/374302
partition 370000/374302
partition 374302/374302
starting epoch: 1/100
batch: 100/1563 - t

batch: 100/1563 - train loss: 11.5483 - test loss: 14.4987 - train acc: 0.4485 - test acc: 0.3596 - 5m 40s
batch: 200/1563 - train loss: 11.8536 - test loss: 14.2795 - train acc: 0.4372 - test acc: 0.3688 - 5m 45s
batch: 300/1563 - train loss: 11.9146 - test loss: 14.1461 - train acc: 0.4300 - test acc: 0.3701 - 5m 51s
batch: 400/1563 - train loss: 11.8092 - test loss: 14.7023 - train acc: 0.4435 - test acc: 0.3617 - 5m 56s
batch: 500/1563 - train loss: 12.1262 - test loss: 14.2434 - train acc: 0.4322 - test acc: 0.3691 - 6m 1s
batch: 600/1563 - train loss: 11.9621 - test loss: 14.6337 - train acc: 0.4425 - test acc: 0.3552 - 6m 6s
batch: 700/1563 - train loss: 12.0691 - test loss: 14.9162 - train acc: 0.4222 - test acc: 0.3575 - 6m 12s
batch: 800/1563 - train loss: 11.9389 - test loss: 14.6133 - train acc: 0.4338 - test acc: 0.3497 - 6m 17s
batch: 900/1563 - train loss: 11.9853 - test loss: 14.9499 - train acc: 0.4400 - test acc: 0.3540 - 6m 22s
batch: 1000/1563 - train loss: 12.0297 

...

batch: 1500/1563 - train loss: 4.9460 - test loss: 15.8154 - train acc: 0.7341 - test acc: 0.4289 - 18m 34s
batch: 1563/1563 - train loss: 5.1564 - test loss: 15.9478 - train acc: 0.7257 - test acc: 0.4223 - 18m 39s
GPU memory used: 7.24 GB - max: 7.56 GB - memory reserved: 7.62 GB - max: 7.62 GB
starting epoch: 14/100
batch: 100/1563 - train loss: 2.7470 - test loss: 16.3441 - train acc: 0.8428 - test acc: 0.4285 - 18m 44s
batch: 200/1563 - train loss: 2.7951 - test loss: 17.0944 - train acc: 0.8497 - test acc: 0.4148 - 18m 49s
batch: 300/1563 - train loss: 3.0584 - test loss: 17.0704 - train acc: 0.8371 - test acc: 0.4216 - 18m 55s
batch: 400/1563 - train loss: 3.2474 - test loss: 16.9926 - train acc: 0.8249 - test acc: 0.4296 - 19m 0s
batch: 500/1563 - train loss: 3.4730 - test loss: 16.6137 - train acc: 0.8065 - test acc: 0.4256 - 19m 6s
batch: 600/1563 - train loss: 3.4379 - test loss: 16.7765 - train acc: 0.8037 - test acc: 0.4221 - 19m 12s
batch: 700/1563 - train loss: 3.6643 - 

FisherPartitioner: param: 512 - partition: 30 - nº part: 18 - block updates: 18
FisherPartitioner: param: 512 - partition: 30 - nº part: 18 - block updates: 18
FisherPartitioner: param: 51200 - partition: 30 - nº part: 1707 - block updates: 360
FisherPartitioner: param: 100 - partition: 30 - nº part: 4 - block updates: 4
total partitions: 374302 - effective block updates: 7782
initializing buffers and inverses...
partition 1/374302
partition 10000/374302
partition 20000/374302
partition 30000/374302
partition 40000/374302
partition 50000/374302
partition 60000/374302
partition 70000/374302
partition 80000/374302
partition 90000/374302
partition 100000/374302
partition 110000/374302
partition 120000/374302
partition 130000/374302
partition 140000/374302
partition 150000/374302
partition 160000/374302
partition 170000/374302
partition 180000/374302
partition 190000/374302
partition 200000/374302
partition 210000/374302
partition 220000/374302
partition 230000/374302
partition 240000/3743

batch: 1400/1563 - train loss: 13.1178 - test loss: 14.1043 - train acc: 0.3990 - test acc: 0.3626 - 5m 31s
batch: 1500/1563 - train loss: 13.1247 - test loss: 14.4040 - train acc: 0.3925 - test acc: 0.3528 - 5m 36s
batch: 1563/1563 - train loss: 13.0547 - test loss: 14.6232 - train acc: 0.3928 - test acc: 0.3491 - 5m 41s
GPU memory used: 7.24 GB - max: 7.56 GB - memory reserved: 7.62 GB - max: 7.62 GB
starting epoch: 5/100
batch: 100/1563 - train loss: 11.5288 - test loss: 14.1530 - train acc: 0.4478 - test acc: 0.3724 - 5m 47s
batch: 200/1563 - train loss: 11.5956 - test loss: 15.3814 - train acc: 0.4453 - test acc: 0.3316 - 5m 52s
batch: 300/1563 - train loss: 11.9684 - test loss: 14.4792 - train acc: 0.4332 - test acc: 0.3613 - 5m 57s
batch: 400/1563 - train loss: 11.7726 - test loss: 15.0461 - train acc: 0.4475 - test acc: 0.3561 - 6m 3s
batch: 500/1563 - train loss: 11.8725 - test loss: 14.6411 - train acc: 0.4385 - test acc: 0.3639 - 6m 8s
batch: 600/1563 - train loss: 11.9369 -

batch: 600/1563 - train loss: 7.6263 - test loss: 13.6577 - train acc: 0.6068 - test acc: 0.4148 - 12m 1s
batch: 700/1563 - train loss: 7.5079 - test loss: 14.1026 - train acc: 0.6156 - test acc: 0.4149 - 12m 7s
batch: 800/1563 - train loss: 7.8148 - test loss: 13.9718 - train acc: 0.6153 - test acc: 0.4188 - 12m 13s
batch: 900/1563 - train loss: 7.8648 - test loss: 14.2381 - train acc: 0.5984 - test acc: 0.4149 - 12m 18s
batch: 1000/1563 - train loss: 7.9081 - test loss: 13.5798 - train acc: 0.6063 - test acc: 0.4183 - 12m 24s
batch: 1100/1563 - train loss: 7.7278 - test loss: 13.7997 - train acc: 0.6112 - test acc: 0.4235 - 12m 29s
batch: 1200/1563 - train loss: 7.7515 - test loss: 14.1103 - train acc: 0.6100 - test acc: 0.4114 - 12m 35s
batch: 1300/1563 - train loss: 7.9826 - test loss: 14.1105 - train acc: 0.5972 - test acc: 0.4216 - 12m 40s
batch: 1400/1563 - train loss: 8.3468 - test loss: 14.3711 - train acc: 0.5787 - test acc: 0.4107 - 12m 46s
batch: 1500/1563 - train loss: 8.2

batch: 1500/1563 - train loss: 4.8618 - test loss: 15.8962 - train acc: 0.7359 - test acc: 0.4271 - 18m 39s
batch: 1563/1563 - train loss: 5.1462 - test loss: 16.4639 - train acc: 0.7285 - test acc: 0.4069 - 18m 44s
GPU memory used: 7.24 GB - max: 7.56 GB - memory reserved: 7.62 GB - max: 7.62 GB
starting epoch: 14/100
batch: 100/1563 - train loss: 2.9321 - test loss: 15.9505 - train acc: 0.8356 - test acc: 0.4329 - 18m 49s
batch: 200/1563 - train loss: 3.0128 - test loss: 16.4292 - train acc: 0.8296 - test acc: 0.4265 - 18m 55s
batch: 300/1563 - train loss: 3.0304 - test loss: 16.5596 - train acc: 0.8337 - test acc: 0.4287 - 19m 0s
batch: 400/1563 - train loss: 3.0562 - test loss: 16.5810 - train acc: 0.8350 - test acc: 0.4219 - 19m 5s
batch: 500/1563 - train loss: 3.4083 - test loss: 16.4013 - train acc: 0.8118 - test acc: 0.4371 - 19m 11s
batch: 600/1563 - train loss: 3.5625 - test loss: 16.8888 - train acc: 0.7965 - test acc: 0.4215 - 19m 16s
batch: 700/1563 - train loss: 3.5852 - 

## testing baseline

In [27]:
for _ in range(nruns):

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    default_metrics, _ = train_network_fisher_optimization(apply_fisher = False,
                                                           net_params = {'p': 0.1},
                                                           epochs = 100,
                                                           time_limit_secs = 1200)

    results_list.append( (default_metrics, buffer_size, partition_size, block_updates) )
    results_list_to_json(results_list, step=step_i)
    step_i += 1
    
    print()

generating CIFAR100 data with 100 classes
Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]  

batch: 1400/1563 - train loss: 16.5994 - test loss: 16.3922 - train acc: 0.2650 - test acc: 0.2778 - 2m 18s
batch: 1500/1563 - train loss: 16.6086 - test loss: 16.2864 - train acc: 0.2759 - test acc: 0.2772 - 2m 24s
batch: 1563/1563 - train loss: 16.0743 - test loss: 17.1867 - train acc: 0.2912 - test acc: 0.2572 - 2m 28s
GPU memory used: 0.13 GB - max: 7.56 GB - memory reserved: 0.23 GB - max: 7.62 GB
starting epoch: 3/100
batch: 100/1563 - train loss: 15.4027 - test loss: 16.0116 - train acc: 0.2919 - test acc: 0.2940 - 2m 32s
batch: 200/1563 - train loss: 14.9696 - test loss: 16.0886 - train acc: 0.3199 - test acc: 0.2873 - 2m 37s
batch: 300/1563 - train loss: 15.0037 - test loss: 16.4906 - train acc: 0.3256 - test acc: 0.2757 - 2m 42s
batch: 400/1563 - train loss: 15.0565 - test loss: 16.3459 - train acc: 0.3187 - test acc: 0.2868 - 2m 47s
batch: 500/1563 - train loss: 15.2491 - test loss: 15.8935 - train acc: 0.3115 - test acc: 0.2981 - 2m 51s
batch: 600/1563 - train loss: 15.0938

batch: 600/1563 - train loss: 9.6747 - test loss: 13.5474 - train acc: 0.5284 - test acc: 0.4057 - 8m 3s
batch: 700/1563 - train loss: 9.7440 - test loss: 14.6558 - train acc: 0.5168 - test acc: 0.3814 - 8m 7s
batch: 800/1563 - train loss: 9.5898 - test loss: 14.0397 - train acc: 0.5225 - test acc: 0.3918 - 8m 12s
batch: 900/1563 - train loss: 9.6970 - test loss: 19.5851 - train acc: 0.5213 - test acc: 0.2809 - 8m 17s
batch: 1000/1563 - train loss: 9.8432 - test loss: 14.2269 - train acc: 0.5185 - test acc: 0.3866 - 8m 22s
batch: 1100/1563 - train loss: 9.7207 - test loss: 14.2599 - train acc: 0.5343 - test acc: 0.3798 - 8m 27s
batch: 1200/1563 - train loss: 9.6413 - test loss: 13.2499 - train acc: 0.5247 - test acc: 0.4130 - 8m 32s
batch: 1300/1563 - train loss: 9.5846 - test loss: 14.5653 - train acc: 0.5294 - test acc: 0.3806 - 8m 37s
batch: 1400/1563 - train loss: 9.8692 - test loss: 13.3567 - train acc: 0.5250 - test acc: 0.4141 - 8m 41s
batch: 1500/1563 - train loss: 9.7465 - tes

batch: 1500/1563 - train loss: 6.1555 - test loss: 15.1252 - train acc: 0.6750 - test acc: 0.4201 - 13m 57s
batch: 1563/1563 - train loss: 6.0714 - test loss: 15.6622 - train acc: 0.6738 - test acc: 0.4047 - 14m 1s
GPU memory used: 0.13 GB - max: 7.56 GB - memory reserved: 0.23 GB - max: 7.62 GB
starting epoch: 12/100
batch: 100/1563 - train loss: 3.5342 - test loss: 15.8188 - train acc: 0.8106 - test acc: 0.4111 - 14m 6s
batch: 200/1563 - train loss: 3.5778 - test loss: 15.9947 - train acc: 0.8041 - test acc: 0.4138 - 14m 11s
batch: 300/1563 - train loss: 3.8025 - test loss: 16.2076 - train acc: 0.7856 - test acc: 0.4090 - 14m 16s
batch: 400/1563 - train loss: 4.0508 - test loss: 15.4642 - train acc: 0.7828 - test acc: 0.4238 - 14m 21s
batch: 500/1563 - train loss: 3.8725 - test loss: 16.0506 - train acc: 0.7871 - test acc: 0.4183 - 14m 26s
batch: 600/1563 - train loss: 4.1937 - test loss: 16.5637 - train acc: 0.7609 - test acc: 0.3951 - 14m 30s
batch: 700/1563 - train loss: 4.3481 - 

batch: 700/1563 - train loss: 1.9249 - test loss: 18.7599 - train acc: 0.8973 - test acc: 0.4172 - 19m 47s
batch: 800/1563 - train loss: 2.1370 - test loss: 19.2206 - train acc: 0.8775 - test acc: 0.4088 - 19m 52s
batch: 900/1563 - train loss: 2.0806 - test loss: 18.9534 - train acc: 0.8829 - test acc: 0.4244 - 19m 57s
time is up! finishing training
batch: 904/1563 - train loss: 2.1181 - test loss: 18.8514 - train acc: 0.8810 - test acc: 0.4282 - 20m 1s
GPU memory used: 0.13 GB - max: 7.56 GB - memory reserved: 0.23 GB - max: 7.62 GB

generating CIFAR100 data with 100 classes
Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4

...

batch: 1000/1563 - train loss: 12.2155 - test loss: 14.6880 - train acc: 0.4241 - test acc: 0.3519 - 5m 35s
batch: 1100/1563 - train loss: 11.7200 - test loss: 15.8874 - train acc: 0.4441 - test acc: 0.3116 - 5m 39s
batch: 1200/1563 - train loss: 12.0340 - test loss: 14.0388 - train acc: 0.4323 - test acc: 0.3726 - 5m 44s
batch: 1300/1563 - train loss: 11.9702 - test loss: 13.9820 - train acc: 0.4385 - test acc: 0.3744 - 5m 49s
batch: 1400/1563 - train loss: 11.9935 - test loss: 14.0284 - train acc: 0.4375 - test acc: 0.3761 - 5m 54s
batch: 1500/1563 - train loss: 11.8194 - test loss: 13.9264 - train acc: 0.4468 - test acc: 0.3796 - 5m 58s
batch: 1563/1563 - train loss: 11.8401 - test loss: 13.8538 - train acc: 0.4481 - test acc: 0.3778 - 6m 2s
GPU memory used: 0.13 GB - max: 7.56 GB - memory reserved: 0.23 GB - max: 7.62 GB
starting epoch: 6/100
batch: 100/1563 - train loss: 10.0771 - test loss: 13.5474 - train acc: 0.5122 - test acc: 0.3996 - 6m 7s
batch: 200/1563 - train loss: 10.48

batch: 200/1563 - train loss: 5.2725 - test loss: 15.3072 - train acc: 0.7203 - test acc: 0.3990 - 11m 8s
batch: 300/1563 - train loss: 5.7252 - test loss: 14.6290 - train acc: 0.7035 - test acc: 0.4201 - 11m 13s
batch: 400/1563 - train loss: 5.8659 - test loss: 14.5531 - train acc: 0.6885 - test acc: 0.4136 - 11m 18s
batch: 500/1563 - train loss: 5.9927 - test loss: 15.1896 - train acc: 0.6735 - test acc: 0.4071 - 11m 22s
batch: 600/1563 - train loss: 6.1284 - test loss: 16.0756 - train acc: 0.6779 - test acc: 0.3814 - 11m 27s
batch: 700/1563 - train loss: 5.9910 - test loss: 15.3739 - train acc: 0.6797 - test acc: 0.3982 - 11m 32s
batch: 800/1563 - train loss: 6.3661 - test loss: 14.9582 - train acc: 0.6591 - test acc: 0.4166 - 11m 36s
batch: 900/1563 - train loss: 6.4443 - test loss: 14.6246 - train acc: 0.6669 - test acc: 0.4149 - 11m 41s
batch: 1000/1563 - train loss: 6.4627 - test loss: 14.9518 - train acc: 0.6616 - test acc: 0.4064 - 11m 46s
batch: 1100/1563 - train loss: 6.9864

batch: 1100/1563 - train loss: 3.3705 - test loss: 17.4778 - train acc: 0.8130 - test acc: 0.4207 - 16m 50s
batch: 1200/1563 - train loss: 3.7729 - test loss: 18.1239 - train acc: 0.7875 - test acc: 0.4077 - 16m 54s
batch: 1300/1563 - train loss: 3.5985 - test loss: 17.1152 - train acc: 0.7956 - test acc: 0.4163 - 16m 59s
batch: 1400/1563 - train loss: 3.6752 - test loss: 17.1755 - train acc: 0.7890 - test acc: 0.4188 - 17m 4s
batch: 1500/1563 - train loss: 3.8501 - test loss: 17.6786 - train acc: 0.7890 - test acc: 0.4046 - 17m 9s
batch: 1563/1563 - train loss: 3.9987 - test loss: 17.6662 - train acc: 0.7713 - test acc: 0.4165 - 17m 13s
GPU memory used: 0.13 GB - max: 7.56 GB - memory reserved: 0.23 GB - max: 7.62 GB
starting epoch: 15/100
batch: 100/1563 - train loss: 2.3083 - test loss: 17.4111 - train acc: 0.8731 - test acc: 0.4234 - 17m 17s
batch: 200/1563 - train loss: 2.1467 - test loss: 17.3637 - train acc: 0.8788 - test acc: 0.4211 - 17m 22s
batch: 300/1563 - train loss: 2.199

batch: 100/1563 - train loss: 26.1293 - test loss: 25.0812 - train acc: 0.0445 - test acc: 0.0566 - 0m 1s
batch: 200/1563 - train loss: 24.1026 - test loss: 23.2340 - train acc: 0.0614 - test acc: 0.0750 - 0m 6s
batch: 300/1563 - train loss: 23.1988 - test loss: 23.9935 - train acc: 0.0774 - test acc: 0.0830 - 0m 11s
batch: 400/1563 - train loss: 22.2593 - test loss: 21.6409 - train acc: 0.1049 - test acc: 0.1124 - 0m 15s
batch: 500/1563 - train loss: 21.8189 - test loss: 21.2069 - train acc: 0.1109 - test acc: 0.1310 - 0m 20s
batch: 600/1563 - train loss: 21.3838 - test loss: 21.0962 - train acc: 0.1231 - test acc: 0.1305 - 0m 24s
batch: 700/1563 - train loss: 20.9898 - test loss: 20.5852 - train acc: 0.1340 - test acc: 0.1405 - 0m 29s
batch: 800/1563 - train loss: 20.7685 - test loss: 20.3266 - train acc: 0.1419 - test acc: 0.1475 - 0m 34s
batch: 900/1563 - train loss: 20.2141 - test loss: 20.8720 - train acc: 0.1522 - test acc: 0.1429 - 0m 39s
batch: 1000/1563 - train loss: 20.0835 

batch: 1000/1563 - train loss: 11.8512 - test loss: 14.2027 - train acc: 0.4428 - test acc: 0.3662 - 5m 38s
batch: 1100/1563 - train loss: 12.2654 - test loss: 14.5043 - train acc: 0.4288 - test acc: 0.3593 - 5m 43s
batch: 1200/1563 - train loss: 11.8573 - test loss: 15.6651 - train acc: 0.4263 - test acc: 0.3259 - 5m 48s
batch: 1300/1563 - train loss: 12.1434 - test loss: 15.8447 - train acc: 0.4341 - test acc: 0.3165 - 5m 52s
batch: 1400/1563 - train loss: 11.8809 - test loss: 14.2387 - train acc: 0.4463 - test acc: 0.3768 - 5m 57s
batch: 1500/1563 - train loss: 12.4886 - test loss: 13.5532 - train acc: 0.4238 - test acc: 0.3895 - 6m 1s
batch: 1563/1563 - train loss: 12.1553 - test loss: 13.7930 - train acc: 0.4316 - test acc: 0.3846 - 6m 5s
GPU memory used: 0.13 GB - max: 7.56 GB - memory reserved: 0.23 GB - max: 7.62 GB
starting epoch: 6/100
batch: 100/1563 - train loss: 10.1557 - test loss: 13.5565 - train acc: 0.5165 - test acc: 0.3932 - 6m 10s
batch: 200/1563 - train loss: 10.28